In [1]:
import pandas as pd

from findlay2025a import core
from findlay2025a.constants import Experiments as Exps
from findlay2025a.constants import Files

In [2]:
exclusions = [
    (
        "CNPIX5-Alessandro",
        Exps.NOD,
        ("early_ext_wake", "late_ext_wake", "ext_wake", "early_ext", "late_ext"),
    ),
    (
        "CNPIX6-Eugene",
        Exps.NOD,
        (
            "early_rec_nrem_match",
            "early_ext_wake",
            "late_ext_wake",
            "ext_wake",
            "early_ext",
            "late_ext",
        ),
    ),
    (
        "CNPIX10-Charles",
        Exps.NOD,
        ("early_ext_wake", "late_ext_wake", "ext_wake", "early_ext", "late_ext"),
    ),
    (
        "CNPIX11-Adrian",
        Exps.NOD,
        ("early_ext_wake", "late_ext_wake", "ext_wake", "early_ext", "late_ext"),
    ),
    (
        "CNPIX11-Adrian",
        Exps.COW,
        ("early_ext_wake", "late_ext_wake", "ext_wake", "early_ext", "late_ext"),
    ),
    (
        "CNPIX17-Hans",
        Exps.COW,
        ("early_ext_wake", "late_ext_wake", "ext_wake", "early_ext", "late_ext"),
    ),
]

measure_display_names = {
    "cx_mean_zlog_delta": "Cortical.SWA",
    "spw_rate_rel2total": "SPW.Rate",
    "spw_mean_zlog_amp": "SPW.Amplitude",
    "spw_mean_zlog_duration": "SPW.Duration",
    "ripple_rate_rel2total": "Ripple.Rate",
    "ripple_mean_zlog_amp": "Ripple.Amplitude",
    "ripple_mean_zlog_duration": "Ripple.Duration",
    "ripple_mean_zfreq": "Ripple.Frequency",
    "ripple_mean_freq": "Ripple.Frequency.Hz",
    "dspk_rate_rel2total": "DSPK.Rate",
    "dspk_mean_zlog_height": "DSPK.Amplitude",
    "hipp_mean_tdr": "Hippocampal.ThetaDeltaRatio",
    "hipp_total_tdr": "Total.Hippocampal.ThetaDeltaRatio",
    "hipp_mean_zlog_theta": "Hippocampal.Theta",
    "hipp_total_zlog_theta": "Total.Hippocampal.Theta",
    "mean_zemg": "EMG",
    "total_zemg": "Total.EMG",
    "cx_mean_zlog_eta": "Cortical.Eta",
    "cxsp_rate_rel2total": "Cortical.Spindle.Rate",
    "cxsp_mean_zlog_Duration": "Cortical.Spindle.Duration",
    "cxsp_mean_zlog_Amplitude": "Cortical.Spindle.Amplitude",
    "cxsp_mean_zlog_AbsPower": "Cortical.Spindle.Power",
    "cxsp_mean_Frequency": "Cortical.Spindle.Frequency",
    "cxsp_mean_zlog_Oscillations": "Cortical.Spindle.Oscillations",
}

In [3]:
def apply_exclusions(df: pd.DataFrame) -> pd.DataFrame:
    essential_cols = ["subject", "experiment", "condition"]
    assert all(col in df.columns for col in essential_cols), (
        f"Missing one or more essential columns: {essential_cols}"
    )

    for sub, exp, conds in exclusions:
        drop = (
            (df["subject"] == sub)
            & (df["experiment"] == exp)
            & (df["condition"].isin(conds))
        )
        df = df[~drop]

    return df.reset_index(drop=True)


def use_display_names(df: pd.DataFrame, minimize: bool = True) -> pd.DataFrame:
    essential_cols = ["subject", "experiment", "condition"]
    assert all(col in df.columns for col in essential_cols), (
        f"Missing one or more essential columns: {essential_cols}"
    )

    if minimize:
        df = df[df["condition"].isin(core.CONDITION_DISPLAY_NAMES)]
        df = df[df["experiment"].isin(core.EXPERIMENT_DISPLAY_NAMES)]
        df = df[essential_cols + list(measure_display_names)]

    df["condition"] = df["condition"].map(core.CONDITION_DISPLAY_NAMES)
    df["experiment"] = df["experiment"].map(core.EXPERIMENT_DISPLAY_NAMES)
    df.rename(columns=measure_display_names, inplace=True)

    return df.reset_index(drop=True)


def get_essential_measures() -> pd.DataFrame:
    nb = core.get_project("seahorse")
    df = pd.read_parquet(
        nb.get_project_file(Files.COMBINED_CONDITION_MEASURES)
    ).reset_index()
    return use_display_names(apply_exclusions(df))

In [4]:
nb = core.get_project("seahorse")
df = get_essential_measures()
df.to_parquet(nb.get_project_file(Files.EXPORTED_CONDITION_MEASURES))